In [1]:
import pandas as pd
import numpy as np

In [ ]:
def find_voltages(data: np.ndarray, v: float) -> np.ndarray:
    '''
    given a matrix of beamline centroid data, with columns of beamline scans at different actuator voltages and rows of slit positions, calculate the necessary voltages to achive the target centroid position.

    Parameters:
    data: np.ndarray
        matrix of beamline centroid data, with rows of different slit positions and columns of pencil beam scans at different actuator voltages
    v: float
        voltage increment

    Returns:
    np.ndarray
        array of voltages required to move the centroid of each pencil beam scan to the target position
    '''
    responses = np.diff(data, axis=1) # calculate the response of each actuator by subtracting previous pencil beam 

    H = responses / v #response per unit charge
    H = np.hstack((np.ones((H.shape[0],1)),H)) #add columns of 1's to the left of H
    H_inv = np.linalg.pinv(H) # calculate the Moore-Penrose pseudo inverse of H


    target = np.mean(data[:,0])
    Y = target - data[:,0] #difference between target and first pencil beam scan where no voltage incrememnt is applied

    voltage_corrections = np.matmul(H_inv,Y) #calculate the voltage required to move the centroid to the target position

    return voltage_corrections[1:] #return the voltages required to move the centroid of each pencil beam scan to the target position

In [149]:
data = pd.read_csv('./I07-HFM-ActiveArea-FocusInEH2-09Apr24-PencilScans.txt', sep = '\t')
data

,mbs1xcentre,#510535,#510536,#510537,#510538,#510539,#510540,#510541,#510542,#510543
0,mm,pix,pix,pix,pix,pix,pix,pix,pix,pix
1,-1.1402,175.61,175.24,175.57,177.31,178.92,180.19,184.93,191.68,197.44
2,-1.13,170.02,169.99,169.95,171.1,173.06,174.68,178.9,186.37,192.31
3,-1.1202,170.46,170.47,170.28,170.21,171.24,172.05,176.92,184.59,190.35
4,-1.11,171.4,171.18,171.14,170.91,171.46,172.97,177.45,185.36,190.91
...,...,...,...,...,...,...,...,...,...,...
150,0.35,245.79,237.54,218.9,204.64,197.94,193.14,188.57,187.32,187.51
151,0.3598,242.35,232.93,214.38,198.9,193.65,189.08,184.08,183.28,182.95
152,0.3698,243.33,232.85,214.74,199.99,193.82,189.3,184.41,183.57,183.5
153,0.3798,249.38,237.48,220.36,205.49,200.41,195.46,190.42,189.83,189.53


In [150]:
info = pd.read_csv('./I07-HFM-ActiveArea-FocusInEH2-09Apr24-Voltages.txt', sep = '\t')
info

,Electrode,Initial best estimate,Best voltage corrections from multiple linear regression,Best corrected voltages
0,NaN,V,V,V
1,1.0,-68,-103.857086224237,36
2,2.0,155,7.85129565214234,147
3,3.0,95,-38.1015907790125,133
4,4.0,18,-21.6964864513985,40
5,5.0,102,84.5319596847066,17
6,6.0,-109,-53.2794543060915,-56
7,7.0,-8,67.0160210654249,-75
8,8.0,52,-2.39956302520734,54


In [151]:
voltage_cols = data.columns[data.columns.str.contains('#')]
data[voltage_cols]

,#510535,#510536,#510537,#510538,#510539,#510540,#510541,#510542,#510543
0,pix,pix,pix,pix,pix,pix,pix,pix,pix
1,175.61,175.24,175.57,177.31,178.92,180.19,184.93,191.68,197.44
2,170.02,169.99,169.95,171.1,173.06,174.68,178.9,186.37,192.31
3,170.46,170.47,170.28,170.21,171.24,172.05,176.92,184.59,190.35
4,171.4,171.18,171.14,170.91,171.46,172.97,177.45,185.36,190.91
...,...,...,...,...,...,...,...,...,...
150,245.79,237.54,218.9,204.64,197.94,193.14,188.57,187.32,187.51
151,242.35,232.93,214.38,198.9,193.65,189.08,184.08,183.28,182.95
152,243.33,232.85,214.74,199.99,193.82,189.3,184.41,183.57,183.5
153,249.38,237.48,220.36,205.49,200.41,195.46,190.42,189.83,189.53


In [152]:
responses = data.loc[1:, voltage_cols].to_numpy().astype(float)
responses

array([[175.61, 175.24, 175.57, ..., 184.93, 191.68, 197.44],
       [170.02, 169.99, 169.95, ..., 178.9 , 186.37, 192.31],
       [170.46, 170.47, 170.28, ..., 176.92, 184.59, 190.35],
       ...,
       [243.33, 232.85, 214.74, ..., 184.41, 183.57, 183.5 ],
       [249.38, 237.48, 220.36, ..., 190.42, 189.83, 189.53],
       [258.62, 246.42, 228.74, ..., 199.99, 199.17, 198.91]])

In [167]:
v = -100
find_voltages(responses, v)

[[-3.700e-01  3.300e-01  1.740e+00 ...  4.740e+00  6.750e+00  5.760e+00]
 [-3.000e-02 -4.000e-02  1.150e+00 ...  4.220e+00  7.470e+00  5.940e+00]
 [ 1.000e-02 -1.900e-01 -7.000e-02 ...  4.870e+00  7.670e+00  5.760e+00]
 ...
 [-1.048e+01 -1.811e+01 -1.475e+01 ... -4.890e+00 -8.400e-01 -7.000e-02]
 [-1.190e+01 -1.712e+01 -1.487e+01 ... -5.040e+00 -5.900e-01 -3.000e-01]
 [-1.220e+01 -1.768e+01 -1.385e+01 ... -4.740e+00 -8.200e-01 -2.600e-01]]
[[ 1.000e+00  3.700e-03 -3.300e-03 ... -4.740e-02 -6.750e-02 -5.760e-02]
 [ 1.000e+00  3.000e-04  4.000e-04 ... -4.220e-02 -7.470e-02 -5.940e-02]
 [ 1.000e+00 -1.000e-04  1.900e-03 ... -4.870e-02 -7.670e-02 -5.760e-02]
 ...
 [ 1.000e+00  1.048e-01  1.811e-01 ...  4.890e-02  8.400e-03  7.000e-04]
 [ 1.000e+00  1.190e-01  1.712e-01 ...  5.040e-02  5.900e-03  3.000e-03]
 [ 1.000e+00  1.220e-01  1.768e-01 ...  4.740e-02  8.200e-03  2.600e-03]]


array([   3.85708622, -107.85129565,  -61.89840922,  -78.30351355,
       -184.53195968,  -46.72054569, -167.01602107,  -97.60043697])

In [162]:
a = np.array([[1,2,3,4,5], [1,2,3,4,5], [1,2,3,4,5]])
a

array([[1, 2, 3, 4, 5],
       [1, 2, 3, 4, 5],
       [1, 2, 3, 4, 5]])

In [171]:
np.mean(a[:,0])

np.float64(1.0)

In [9]:
v=-100
influence_funcs = np.diff(responses, axis=1) / v # effect of each electrode on the beam position
influence_funcs

array([[ 3.700e-03, -3.300e-03, -1.740e-02, ..., -4.740e-02, -6.750e-02,
        -5.760e-02],
       [ 3.000e-04,  4.000e-04, -1.150e-02, ..., -4.220e-02, -7.470e-02,
        -5.940e-02],
       [-1.000e-04,  1.900e-03,  7.000e-04, ..., -4.870e-02, -7.670e-02,
        -5.760e-02],
       ...,
       [ 1.048e-01,  1.811e-01,  1.475e-01, ...,  4.890e-02,  8.400e-03,
         7.000e-04],
       [ 1.190e-01,  1.712e-01,  1.487e-01, ...,  5.040e-02,  5.900e-03,
         3.000e-03],
       [ 1.220e-01,  1.768e-01,  1.385e-01, ...,  4.740e-02,  8.200e-03,
         2.600e-03]])

In [10]:
find_voltages(responses, -100, 0.1)

array([ -889.39922782,  -886.19134255,   365.83309381, -1605.35671047,
         882.85645969, -1547.49491428,  3223.60634806,  -534.37217805])

In [6]:
data = pd.read_csv('./test/I07-VFM-ActiveArea-FocusInEH2-09Apr24-PencilScans.txt', sep = '\t')
data

,mbs1ycentre,#510511,#510512,#510513,#510514,#510515,#510516,#510517,#510518,#510519,#510520,#510521,#510522,#510523,#510524,#510525,#510526,#510527
0,mm,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix
1,0.5302,277.8,276.61,276.91,277.28,277.4,277.71,276.76,275.23,272.56,270.4,265.6,260.16,252.98,245.75,239.46,233.79,230.05
2,0.5401,272.64,270.86,271.68,271.99,272.12,272.25,271.15,269.93,267.14,264.95,260.22,254.62,247.55,240.05,233.86,228.45,225.02
3,0.5501,272.99,272.16,272.51,272.83,272.95,273.05,272.17,270.52,267.78,265.7,261.1,255.44,247.91,240.89,234.48,229.69,226.67
4,0.5601,275.42,274.28,275,275.1,275.15,275.26,274.39,272.79,270.12,267.89,263.38,257.59,250.45,243.54,236.89,231.73,230.36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,1.6501,226.99,227.83,230.54,234.28,236.51,240.68,243.69,247.12,249.2,250.81,251.9,251.57,252.43,251.37,250.58,251.52,252.02
114,1.6601,229.35,230.88,233.09,236.91,238.71,242.79,244.78,248.23,251.16,251.69,253,252.93,253.53,252.82,251.94,252.31,253.24
115,1.6698,231.85,233.08,235.44,238.02,240.51,244.18,245.87,249.56,251.85,252.52,253.81,253.94,254.17,253.36,253.02,253.73,254.24
116,1.6803,235.69,237.92,239.75,241.99,244.15,247.78,249.36,252.76,254.2,255.39,256.56,255.93,256.44,256.08,255.14,256.63,256.91


In [5]:
voltage_cols = data.columns[data.columns.str.contains('voltage')]
data[voltage_cols]

,bimorph_channel_10_voltage_out_readback_value,bimorph_channel_11_voltage_out_readback_value,bimorph_channel_12_voltage_out_readback_value,bimorph_channel_13_voltage_out_readback_value,bimorph_channel_14_voltage_out_readback_value,bimorph_channel_15_voltage_out_readback_value,bimorph_channel_16_voltage_out_readback_value,bimorph_channel_1_voltage_out_readback_value,bimorph_channel_2_voltage_out_readback_value,bimorph_channel_3_voltage_out_readback_value,bimorph_channel_4_voltage_out_readback_value,bimorph_channel_5_voltage_out_readback_value,bimorph_channel_6_voltage_out_readback_value,bimorph_channel_7_voltage_out_readback_value,bimorph_channel_8_voltage_out_readback_value,bimorph_channel_9_voltage_out_readback_value
0,393.0,503.0,487.0,538.0,624.0,676.0,606.0,557.0,550.0,496.0,473.0,481.0,433.0,448.0,480.0,444.0
1,393.0,503.0,487.0,538.0,624.0,676.0,606.0,557.0,550.0,496.0,473.0,481.0,433.0,448.0,480.0,444.0
2,393.0,503.0,487.0,538.0,624.0,676.0,606.0,557.0,550.0,496.0,473.0,481.0,433.0,448.0,480.0,444.0
3,393.0,503.0,487.0,538.0,624.0,676.0,606.0,557.0,550.0,496.0,473.0,481.0,433.0,448.0,480.0,444.0
4,393.0,503.0,487.0,538.0,624.0,676.0,606.0,557.0,550.0,496.0,474.0,481.0,433.0,448.0,480.0,444.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,393.0,503.0,487.0,538.0,624.0,676.0,606.0,657.0,650.0,596.0,573.0,581.0,533.0,548.0,480.0,444.0
124,393.0,503.0,487.0,538.0,624.0,676.0,606.0,657.0,650.0,596.0,573.0,581.0,533.0,548.0,480.0,444.0
125,393.0,503.0,487.0,538.0,624.0,676.0,606.0,657.0,650.0,596.0,573.0,581.0,533.0,548.0,480.0,444.0
126,393.0,503.0,487.0,538.0,624.0,676.0,606.0,657.0,650.0,596.0,573.0,581.0,533.0,548.0,480.0,444.0


In [6]:
for col in voltage_cols:
    print(data[col].value_counts())

bimorph_channel_10_voltage_out_readback_value
393.0    128
Name: count, dtype: int64
bimorph_channel_11_voltage_out_readback_value
503.0    128
Name: count, dtype: int64
bimorph_channel_12_voltage_out_readback_value
487.0    128
Name: count, dtype: int64
bimorph_channel_13_voltage_out_readback_value
538.0    128
Name: count, dtype: int64
bimorph_channel_14_voltage_out_readback_value
624.0    128
Name: count, dtype: int64
bimorph_channel_15_voltage_out_readback_value
676.0    128
Name: count, dtype: int64
bimorph_channel_16_voltage_out_readback_value
606.0    128
Name: count, dtype: int64
bimorph_channel_1_voltage_out_readback_value
657.0    112
557.0     16
Name: count, dtype: int64
bimorph_channel_2_voltage_out_readback_value
650.0    96
550.0    32
Name: count, dtype: int64
bimorph_channel_3_voltage_out_readback_value
596.0    80
496.0    48
Name: count, dtype: int64
bimorph_channel_4_voltage_out_readback_value
573.0    64
473.0    63
474.0     1
Name: count, dtype: int64
bimorph_cha